In [1]:
#| default_exp notebook

In [2]:
#| export
import argparse
import json

from IPython.core.magic import Magics, magics_class, line_magic, cell_magic, line_cell_magic

import py2gift.input_file

## Convenience functions

A class to act as a *container*. This will be useful to simulate a *Python* module.

In [3]:
#| export
class ClassesContainer:
    
    def add(cls, class_to_add):
        
        setattr(cls, class_to_add.__name__, class_to_add)

In [4]:
classes_container = ClassesContainer()

In [5]:
class DummyClass:
    
    n = 3

In [6]:
classes_container.add(DummyClass)

In [7]:
classes_container.DummyClass

__main__.DummyClass

In [8]:
classes_container.DummyClass.n

3

## Magics

In [9]:
#| export
# The class MUST call this class decorator at creation time
@magics_class
class MyMagics(Magics):
    
    statement_key = 'statement'
    feedback_key = 'feedback'
    
    def __init__(self, shell=None,  **kwargs):
        
        super().__init__(shell=shell, **kwargs)
        
        
        self.location_parser = argparse.ArgumentParser(description='Specification')
        self.location_parser.add_argument('settings', help='Settings object')
        self.location_parser.add_argument('-c', '--cls', default=None, help='class')
        
        # the name of a category can contain spaces; notice that this will yield a *list* rather than a string
        self.location_parser.add_argument('-C', '--category', default=None, nargs='+', help='category')
    
    def process(self, line: str, cell: str, key: str):
        
        line_arguments = self.location_parser.parse_args(line.split())
        
#         print(f'line_arguments.category={line_arguments.category}')
        
        if line_arguments.category:
            
            category = json.loads(' '.join(line_arguments.category))
        
        else:
            
            category = None
        
        settings = self.shell.user_ns[line_arguments.settings]
        
        if category is None:
            
            category = settings.store['categories'][-1]['name']
            
        
        cls = line_arguments.cls
        
        if cls is None:
            
            cls = settings.locate(category_name=category)['classes'][-1]['name']
        

#         print(f'category={category}, class={cls}')
        
        settings.locate(category_name=category, class_name=cls)[key] = cell

    @cell_magic
    def statement(self, line, cell):

        self.process(line, cell, self.statement_key)
        
        return f'statement recorded'
    
    @cell_magic
    def feedback(self, line, cell):
        
        self.process(line, cell, self.feedback_key)
        
        return f'feedback recorded'

## Parser

The class can be instantiated directly,

In [10]:
m = MyMagics()

to test the parser,

In [11]:
m.location_parser.parse_args('settings -c myclass -C mycategory'.split())

Namespace(settings='settings', cls='myclass', category=['mycategory'])

When no category nor class are passed:

In [12]:
m.location_parser.parse_args('settings'.split())

Namespace(settings='settings', cls=None, category=None)

Parsed category is always a list

In [13]:
m.location_parser.parse_args('settings -c myc -C cat 1'.split())

Namespace(settings='settings', cls='myc', category=['cat', '1'])

`json` should be used to pass the category. Then, within the *magic*, the `category` list is joined (with a space) and parsed back using `json` to whatever type it originally had.

In [14]:
category = 'foo'
line = m.location_parser.parse_args(f'settings -c myc -C {json.dumps(category)}'.split())
json.loads(' '.join(line.category))

'foo'

In [15]:
category = 'foo foo'
line = m.location_parser.parse_args(f'settings -c myc -C {json.dumps(category)}'.split())
json.loads(' '.join(line.category))

'foo foo'

In [16]:
category = ['cat', 'subcat']
line = m.location_parser.parse_args(f'settings -c myc -C {json.dumps(category)}'.split())
json.loads(' '.join(line.category))

['cat', 'subcat']

Notice that a `json`-*dumped* string is not equal to the string,

In [17]:
json.dumps('Category 1') == 'Category 1'

False

meaning that, if in a variable, the category must **always** be dumped

## Actual use

*Magics* must be somehow registered

In [18]:
#| export

get_ipython().register_magics(MyMagics)

In [19]:
# %lmagic -n foo

In [20]:
settings = py2gift.input_file.Settings()
category_name = settings.add_category('Category 1')
settings.add_or_update_class(category_name=category_name, class_name='question name', question_base_name='base name', n_instances=2)

In [21]:
settings.to_dict()

{'output file': 'quiz.yaml',
 'pictures base directory': 'quiz/pics',
 'categories': [{'name': 'Category 1',
   'classes': [{'name': 'question name',
     'question base name': 'base name',
     'number of instances': 2}]}]}

In [22]:
%%statement settings
Consider

bla bla

'statement recorded'

In [23]:
settings

{'categories': [{'classes': [{'name': 'question name',
                              'number of instances': 2,
                              'question base name': 'base name',
                              'statement': 'Consider\n\nbla bla\n'}],
                 'name': 'Category 1'}],
 'output file': 'quiz.yaml',
 'pictures base directory': 'quiz/pics'}

In [24]:
cls = 'ClassA'
category = 'Category 1'
settings.add_or_update_class(category_name=category, class_name=cls, question_base_name='base name', n_instances=2)

In [25]:
settings

{'categories': [{'classes': [{'name': 'question name',
                              'number of instances': 2,
                              'question base name': 'base name',
                              'statement': 'Consider\n\nbla bla\n'},
                             {'name': 'ClassA',
                              'number of instances': 2,
                              'question base name': 'base name'}],
                 'name': 'Category 1'}],
 'output file': 'quiz.yaml',
 'pictures base directory': 'quiz/pics'}

In [26]:
%%statement settings -c {cls} -C {json.dumps(category)}
more blah

'statement recorded'

In [27]:
cls = 'cls'
category = 'foo foo'

settings = py2gift.input_file.Settings()
category_name = settings.add_category(category)
settings.add_or_update_class(category_name=category_name, class_name=cls, question_base_name='base name', n_instances=2)

In [28]:
%%statement settings -C {json.dumps(category)} -c {cls}
more

'statement recorded'

In [29]:
cls = 'cls'
category = 'wap wap'
base_category = 'oh'

settings = py2gift.input_file.Settings()
category_name = settings.add_category(category_name=category, base_category=base_category)
settings.add_or_update_class(category_name=category_name, class_name=cls, question_base_name='base name', n_instances=2)

In [30]:
category_name

['oh', 'oh/wap wap']

In [31]:
settings

{'categories': [{'classes': [{'name': 'cls',
                              'number of instances': 2,
                              'question base name': 'base name'}],
                 'name': ['oh', 'oh/wap wap']}],
 'output file': 'quiz.yaml',
 'pictures base directory': 'quiz/pics'}

In [32]:
%%feedback  settings -C {json.dumps(category_name)} -c {cls}
We must...

'feedback recorded'

In [33]:
settings

{'categories': [{'classes': [{'feedback': 'We must...\n',
                              'name': 'cls',
                              'number of instances': 2,
                              'question base name': 'base name'}],
                 'name': ['oh', 'oh/wap wap']}],
 'output file': 'quiz.yaml',
 'pictures base directory': 'quiz/pics'}

In [34]:
#| include: false
import nbdev.export
nbdev.export.nbdev_export('60_notebook.ipynb')

Converted 60_notebook.ipynb.
